In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 1. Pré-processar os dados
# Carregue seus datasets balanceados
train_data = pd.read_csv('../data/preprocessed/movies_genres_grouped_train_preprocessed.csv')
test_data = pd.read_csv('../data/preprocessed/movies_genres_grouped_test_preprocessed.csv')

category_columns = train_data.columns.drop(['Name', 'Description', 'Combined'])

# Defina os parâmetros de pré-processamento
max_features = 10000  # Número máximo de palavras a serem usadas (palavras mais frequentes)
maxlen = 200  # Número máximo de palavras no texto

# Tokenize os textos
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['Combined'])

X_train = tokenizer.texts_to_sequences(train_data['Combined'])
X_test = tokenizer.texts_to_sequences(test_data['Combined'])

# Padronize os textos
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Obtenha os labels one-hot encoded
y_train = train_data[category_columns].values
y_test = test_data[category_columns].values

# 2. Construir o modelo
embedding_dim = 128  # Dimensão do vetor de embedding
lstm_units = 64  # Unidades LSTM

model = Sequential([
    Embedding(max_features, embedding_dim, input_length=maxlen),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Bidirectional(LSTM(lstm_units)),
    Dense(len(category_columns), activation='sigmoid')
])

# 3. Treinar o modelo
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# 4. Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

Epoch 1/10
156/156 [==============================] - 127s 749ms/step - loss: 0.4507 - accuracy: 0.3451 - val_loss: 0.4376 - val_accuracy: 0.3478
Epoch 2/10
156/156 [==============================] - 94s 603ms/step - loss: 0.3761 - accuracy: 0.4263 - val_loss: 0.3823 - val_accuracy: 0.4547
Epoch 3/10
156/156 [==============================] - 86s 549ms/step - loss: 0.2811 - accuracy: 0.5673 - val_loss: 0.3949 - val_accuracy: 0.4674
Epoch 4/10
156/156 [==============================] - 87s 558ms/step - loss: 0.2068 - accuracy: 0.6475 - val_loss: 0.4537 - val_accuracy: 0.4583
Epoch 5/10
156/156 [==============================] - 93s 599ms/step - loss: 0.1560 - accuracy: 0.6839 - val_loss: 0.4912 - val_accuracy: 0.4330
Epoch 6/10
156/156 [==============================] - 100s 642ms/step - loss: 0.1225 - accuracy: 0.6956 - val_loss: 0.5672 - val_accuracy: 0.4257
Epoch 7/10
156/156 [==============================] - 92s 592ms/step - loss: 0.0968 - accuracy: 0.7107 - val_loss: 0.6179 - val_

In [2]:
from sklearn.metrics import classification_report

# Prever e converter as previsões para o formato binarizado
y_pred = model.predict(X_test)
y_pred_binarized = np.round(y_pred)

# Calcular e exibir métricas de avaliação
report = classification_report(y_test, y_pred_binarized, target_names=category_columns, zero_division=0)
print(report)

79/79 [==============================] - 16s 182ms/step
                         precision    recall  f1-score   support

                 Comedy       0.57      0.59      0.58       862
                  Crime       0.62      0.40      0.49       452
                  Drama       0.72      0.60      0.65      1416
                Romance       0.42      0.30      0.35       453
   Action and Adventure       0.65      0.52      0.58       833
Documentary and History       0.44      0.31      0.36       267
   Family and Animation       0.52      0.39      0.45       271
     Fantasy and Sci-Fi       0.51      0.34      0.41       320
    Horror and Thriller       0.58      0.45      0.51       577

              micro avg       0.61      0.49      0.54      5451
              macro avg       0.56      0.43      0.49      5451
           weighted avg       0.60      0.49      0.54      5451
            samples avg       0.60      0.51      0.52      5451



In [6]:
from sklearn.metrics import f1_score, average_precision_score, precision_recall_curve, precision_score, recall_score

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
print("F1-score (micro): {:.2f}".format(f1_micro))
print("F1-score (macro): {:.2f}".format(f1_macro))
print("Precision (micro): {:.2f}".format(precision_micro))
print("Precision (macro): {:.2f}".format(precision_macro))
print("Recall (micro): {:.2f}".format(recall_micro))
print("Recall (macro): {:.2f}".format(recall_macro))

y_pred = model.predict(X_test)
f1_scores_per_class = f1_score(y_test, np.round(y_pred), average=None)
auc_pr_per_class = average_precision_score(y_test, y_pred, average=None)
for i, category in enumerate(category_columns):
    print("Class: {}".format(category))
    print("F1-score: {:.2f}".format(f1_scores_per_class[i]))
    print("AUC-PR: {:.2f}".format(auc_pr_per_class[i]))
    print("\n")

79/79 [==============================] - 13s 159ms/step
F1-score (micro): 0.54
F1-score (macro): 0.49
Precision (micro): 0.61
Precision (macro): 0.56
Recall (micro): 0.49
Recall (macro): 0.43
79/79 [==============================] - 13s 159ms/step
Class: Comedy
F1-score: 0.58
AUC-PR: 0.61


Class: Crime
F1-score: 0.49
AUC-PR: 0.52


Class: Drama
F1-score: 0.65
AUC-PR: 0.74


Class: Romance
F1-score: 0.35
AUC-PR: 0.35


Class: Action and Adventure
F1-score: 0.58
AUC-PR: 0.66


Class: Documentary and History
F1-score: 0.36
AUC-PR: 0.33


Class: Family and Animation
F1-score: 0.45
AUC-PR: 0.46


Class: Fantasy and Sci-Fi
F1-score: 0.41
AUC-PR: 0.45


Class: Horror and Thriller
F1-score: 0.51
AUC-PR: 0.54


